# literature review

In [1]:
# ! cd ~/git/openai; pip3 install -e .
# !pip3 install --upgrade openai

In [2]:
from abcli.modules import objects
from openai_commands import NAME, VERSION
from openai_commands import env
from openai_commands.literature_review.functions import review_literature
from openai_commands.logger import logger

logger.info(f"using {NAME}-{VERSION}.")

🛠️  using openai_commands-3.91.1.


In [3]:
# filename = "review_463333_screen_csv_20240730130035.csv"
# filename = "review_463333_irrelevant_csv_20240730131027.csv"
filename = "review_463333_screen_csv_20240730130035.csv"

In [4]:
assert review_literature(
        input_object_name= env.LITERATURE_REVIEW_OBJECT,
        output_object_name=objects.unique_object("notebook-test"),
        filename=filename,
        question_filename="question1.yaml",
        count=5 # set to -1 to process all
)

:: 📂 notebook-test-2024-08-05-12-17-57-59743
🛠️  openai_commands.literature_review.review_literature: AMR-v7/review_463333_screen_csv_20240730130035.csv -question1.yaml-5X-> notebook-test-2024-08-05-12-17-57-59743/review_463333_screen_csv_20240730130035-question1.csv[question1]
:: loaded 3751 row(s) of Title, Authors, Abstract, Published Year, Published Month, Journal, Volume, Issue, Pages, Accession Number, DOI, Ref, Covidence #, Study, Notes, Tags from /Users/kamangir/storage/abcli/AMR-v7/review_463333_screen_csv_20240730130035.csv
🛠️  read this abstract of a scentific paper @ 3 choice(s): {'antimicrobial': 'if it talks about antimicrobial treatment for cholera.', 'susceptibility': 'if it has information regarding antibiotic-reistance or susceptibility to cholerae strains.', 'otherwise': 'not relevant'}
🛠️  prompt: read this abstract of a scentific paper and return the correct choice from the following list followed with "because" and then describe the reason for this choice in less 

In [5]:
# END